In [1]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [26]:
from litellm import completion
from typing import List, Dict
import json
import re


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


what_to_help_with = input("What do you need help with?")

messages = [
    {"role": "system", "content": "You are an expert on python programming.  When given a description of a function, create a python program that meets that description.  When you respond, respond only with a json dictionarey that has one entry for the code and different entries for any commentary"},
    {"role": "user", "content": what_to_help_with}
]

response = generate_response(messages)


What do you need help with?reverse a list


In [27]:
import json
import os

def construct_prompt(prior_messages, agent_response, next_task):
    """append the agent repsonse to the prior messages and add the next task"""
    messages = prior_messages
    messages.append({"role": "assistant", "content": agent_response})
    messages.append({"role": "user", "content": next_task})
    return messages


def parse_response(response):
    """Extracts action and parameters from the LLM response."""
    try:
        json_pattern = r'\{.*?\}'
        # Find the JSON string in the text
        match = re.search(json_pattern, response, re.DOTALL)

        if match:
          json_string = match.group(0)
          try:
              # Parse the JSON string into a Python dictionary
              data = json.loads(json_string)
              print(data)
              action = data["code"]
              parameters = data["commentary"]
              return action, parameters
          except json.JSONDecodeError as e:
              print("Error decoding JSON:", e)
        else:
            print("No JSON content found in the text.")

    except (json.JSONDecodeError, KeyError):
        return None, None



def execute_action(action, parameters):
    """Executes the action with the given parameters."""
    if action == "list_files":
        files = os.listdir(parameters.get("directory", "."))
        return f"Files in directory: {', '.join(files)}"
    elif action == "read_file":
        try:
            with open(parameters["file_path"], "r") as f:
                content = f.read()
            return f"File content: {content}"
        except FileNotFoundError:
            return "File not found."
    # Add more actions as needed
    else:
        return "Invalid action."

def convert_result_to_string(result):
    """Formats the result into a string."""
    return str(result)



In [28]:
def agent_loop(memory, user_input, system_rules):
    """Main agent loop."""

    while True:
        messages = construct_prompt(prior_messages, user_input, system_rules)
        response = generate_response(messages)
        action, parameters = parse_response(response)

        if action is None:
            print("Invalid response format.")
            break

        result = execute_action(action, parameters)
        result_string = convert_result_to_string(result)

        # Update memory, check for termination conditions, etc.
        memory += f"\nAction: {action}, Result: {result_string}"

        if action == "terminate":
            break

        # Get next user input or continue based on logic
        user_input = input("Enter your next query: ")

    print("Agent loop terminated.")

# Example usage
prior_messages = messages
user_input= what_to_help_with
print(response)
agent_response, commentary = parse_response(response)
print(agent_repsonse)
#user_input = "List files in the current directory."
#system_rules = "You can list files, read files, and answer questions about them."

agent_loop(prior_messages, user_input, agent_response)

```json
{
  "code": "def reverse_list(input_list):\n    return input_list[::-1]\n\n# Example usage:\n# my_list = [1, 2, 3, 4, 5]\n# reversed_list = reverse_list(my_list)\n# print(reversed_list)  # Output: [5, 4, 3, 2, 1]",
  "commentary": "This function takes a list as input and returns a new list that is the reverse of the input. It uses Python's slicing feature to reverse the list."
}
```
{'code': 'def reverse_list(input_list):\n    return input_list[::-1]\n\n# Example usage:\n# my_list = [1, 2, 3, 4, 5]\n# reversed_list = reverse_list(my_list)\n# print(reversed_list)  # Output: [5, 4, 3, 2, 1]', 'commentary': "This function takes a list as input and returns a new list that is the reverse of the input. It uses Python's slicing feature to reverse the list."}
def reverse_list(lst):
    return lst[::-1]

# Example usage:
example_list = [1, 2, 3, 4, 5]
reversed_list = reverse_list(example_list)
print(reversed_list)  # Output: [5, 4, 3, 2, 1]
{'code': 'def reverse_list(input_list):\n    r